**Set environment**

In [2]:
source ../config_duke.sh

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation



**Input**

In [3]:
ls -d ${FD_WRK}/count_fragment/*/

/work/kk319/out/CombEffect_STARR/count_fragment/Input/
/work/kk319/out/CombEffect_STARR/count_fragment/Input1/
/work/kk319/out/CombEffect_STARR/count_fragment/Input2/
/work/kk319/out/CombEffect_STARR/count_fragment/Input3/
/work/kk319/out/CombEffect_STARR/count_fragment/Input4/
/work/kk319/out/CombEffect_STARR/count_fragment/Input5/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX2_DMSO/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX2_Dex/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX3_DMSO/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX3_Dex/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX4_DMSO/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX4_Dex/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX5_DMSO/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX5_Dex/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX_DMSO/
/work/kk319/out/CombEffect_STARR/count_fragment/TFX_Dex/


**Output**

In [4]:
ls -d ${FD_WRK}/annotation_fragment/*/

/work/kk319/out/CombEffect_STARR/annotation_fragment/Input/
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input2/
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input3/
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input4/
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input5/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_DMSO/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_Dex/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_DMSO/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_Dex/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_DMSO/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_Dex/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_DMSO/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_Dex/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX_DMSO/
/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX_Dex/


**TEST**

In [5]:
CHROM=chr17
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa
ls ${FD_GEN}/${FN_GEN}

/work/kk319/annotation/genome/hg38/fasta/chr17.fa


**RUN**

In [16]:
source ../config_duke.sh
source ../load_module_bedtools.sh

sbatch -p scavenger \
    --array 0-23 \
    --mem 20G \
    -o ${FD_LOG}/annotate_fragment_chrom.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_WRK}/count_fragment/*/))
FN_BED=${TARGET}.bed

### motif annotation
FD_MTF=${FD_ANN}/motif_cluster_jvierstra/hg38_archetype_motifs_v1
FN_MTF=${CHROM}_rm_mouse_merge.bed.gz

### genome annotation
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; annotate fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_WRK}/annotation_fragment/${SAMPLE}
    FN_OUT=${TARGET}.bed
    #FN_NUC=${TARGET}_nuc.bed
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 3 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### annotation using intersect
    bedtools intersect -a ${FD_BED}/${FN_BED} -b ${FD_MTF}/${FN_MTF} -wo > ${FD_OUT}/${FN_OUT}
    #bedtools nuc -fi ${FD_GEN}/${FN_GEN} -bed ${FD_OUT}/${FN_OUT}        > ${FD_OUT}/${FN_NUC}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    head -n 3 ${FD_OUT}/${FN_OUT}
    echo
    #echo "Show the first few lines of the output file"
    #echo ${FD_OUT}/${FN_NUC}
    #head -n 3 ${FD_OUT}/${FN_NUC}
    #echo
done
echo "Done!"

EOF

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

You are on Duke Server: DCC
Submitted batch job 14905797


**CHECK**

In [12]:
readlink -f ${FD_LOG}/annotate_fragment_chrom.0.txt

/work/kk319/out/CombEffect_STARR/log/annotate_fragment_chrom.0.txt


In [5]:
cat ${FD_LOG}/annotate_fragment_chrom.1.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Slurm Array Index:  1
RUN: loop through samples; annotate fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /work/kk319/out/CombEffect_STARR/count_fragment/Input//chr2.bed
Output file:  /work/kk319/out/CombEffect_STARR/annotation_fragment/Input/chr2.bed

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/count_fragment/Input//chr2.bed
chr2	11223	12149	1
chr2	11224	12142	1
chr2	11234	12150	2

Show the first few lines of the output file
/work/kk319/out/CombEffect_STARR/annotation_fragment/Input/chr2.bed
chr2	11223	12149	1	chr2	10835	11299	GC-tract	7.420186207	76
chr2	11223	12149	1	chr2	11198	11225	NRF1	11.8977	2
chr2	11223	12149	1	chr2	11213	11227	ZKSCAN1	6.

## Get file size

In [1]:
source ../config_duke.sh

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation



In [2]:
ls -lh ${FD_WRK}/annotation_fragment/*/chr1.bed

-rw-r--r--. 1 kk319 root  90G Jul 19 01:34 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input/chr1.bed
-rw-r--r--. 1 kk319 root  19G Jul 19 01:54 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr1.bed
-rw-r--r--. 1 kk319 root  20G Jul 19 02:16 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input2/chr1.bed
-rw-r--r--. 1 kk319 root  23G Jul 19 02:39 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input3/chr1.bed
-rw-r--r--. 1 kk319 root  19G Jul 19 02:59 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input4/chr1.bed
-rw-r--r--. 1 kk319 root  15G Jul 19 03:15 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input5/chr1.bed
-rw-r--r--. 1 kk319 root  41G Jul 19 03:55 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_DMSO/chr1.bed
-rw-r--r--. 1 kk319 root  41G Jul 19 04:36 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_Dex/chr1.bed
-rw-r--r--. 1 kk319 root  26G Jul 19 05:02 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3

In [3]:
ls -lh ${FD_WRK}/annotation_fragment/*/chr17.bed

-rw-r--r--. 1 kk319 root  35G Jul 19 00:25 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input/chr17.bed
-rw-r--r--. 1 kk319 root 7.1G Jul 19 00:30 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr17.bed
-rw-r--r--. 1 kk319 root 7.7G Jul 19 00:35 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input2/chr17.bed
-rw-r--r--. 1 kk319 root 9.0G Jul 19 00:42 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input3/chr17.bed
-rw-r--r--. 1 kk319 root 7.2G Jul 19 00:46 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input4/chr17.bed
-rw-r--r--. 1 kk319 root 5.8G Jul 19 00:50 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input5/chr17.bed
-rw-r--r--. 1 kk319 root  19G Jul 19 01:01 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_DMSO/chr17.bed
-rw-r--r--. 1 kk319 root  19G Jul 19 01:12 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_Dex/chr17.bed
-rw-r--r--. 1 kk319 root  12G Jul 19 01:18 /work/kk319/out/CombEffect_STARR/annotation_fragm